In [2]:
# adf test
!pip install statsmodels

#font
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

#font
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import glob
from sklearn.externals import joblib 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
test = pd.read_excel('/content/drive/MyDrive/빅콘2021/Data/02_평가데이터_update(210806)/2021 빅콘테스트_데이터분석분야_퓨처스리그_ECO제주_평가데이터_210806_update.xlsx',header=[0,1],index_col=[0,1]).fillna(0)
test.head(5)

읍면동별 음식물 쓰레기 월 배출량 읍면동별 음식물 쓰레기 월 배출량                            
                   Unnamed: 2_level_1 # 배출량은 소수점 첫째자리 까지 표기 바랍니다.
NO 행정동명                     7월 배출량(g)                   8월 배출량(g)
1  한림읍                              0                           0
2  애월읍                              0                           0
3  구좌읍                              0                           0
4  조천읍                              0                           0

In [5]:
# 처리해야 하는 동 list
name_list = [i[1] for i in test.index]
name_list.remove('행정동명')

In [6]:
# 원하는 행정동만 뽑기
def extract_area(df:pd.DataFrame,name: str):
  res = df[df['emd_nm'] == name]
  return res

In [7]:
#주어진 데이터가 가져야하는 모든 날짜 추출
from datetime import datetime, timedelta
final = pd.DataFrame()

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("2018-01-01", "2021-06-30")
future = date_range("2021-07-01", "2021-08-31")

In [8]:
def check_df(df:pd.DataFrame, name:str): #없는날짜(결측치), 존재하는 데이터에서 결측치 확인
  flag = True
  df_date = df['base_date'].unique()
  if len(df_date) != len(dates):
    print(f'없는 날짜 {len(dates) - len(df_date)}개 존재')
    flag = True
  else: 
    flag = False

  for i in range(len(list(df.isnull().sum()))):
    if df.isnull().sum()[i] != 0 :
      print(f'{df.isnull().sum().index[i]}에서 결측치{df.isnull().sum()[i]}개 존재')
  
  return flag

In [9]:
# 없는 날짜 뽑기
def is_no_date(df:pd.DataFrame):
  no_date = []

  if check_df(name): #날짜가 없으면 없는 날짜 확인
    for real_date in dates:
      if real_date not in list(df['base_date']):
        no_date.append(real_date)

  return no_date

In [10]:
def fill_null(df:pd.DataFrame):
    date = df.groupby(['base_date']).sum().reset_index() #날짜별로 묶기
    #없는 날짜 채우기
    temp = pd.DataFrame()
    temp['base_date'] = dates
    merge_df = pd.merge(temp,date, how='outer',on='base_date')
    
    #결측치 보간
    merge_df['base_date'] = pd.to_datetime(merge_df['base_date'])
    merge_df.set_index('base_date',inplace=True)

    # date_idx = dates.index(datetime.strptime('2020-01-01', "%Y-%m-%d"))
    date_idx = dates.index('2020-01-01')
    #아예 맨 처음이 nan값인 경우
    if True in list(merge_df.iloc[0,:].isnull()):
        for i in range(len(merge_df.columns)):
            if np.isnan(merge_df.iloc[0,i]):
                try:
                    merge_df.iloc[0,i] = merge_df.iloc['2019-01-01',i]
                except: 
                    merge_df.iloc[0,i] = merge_df.iloc[date_idx,i]

    merge_df = merge_df.astype(float)
    merge_df = merge_df.interpolate(method='time')
    
    print('final',merge_df)
    return merge_df

In [11]:
# main
def main(df:pd.DataFrame,name:str):
    df = extract_area(df,name) #원하는 동만 뽑기
    if check_df(df, name): #True면 없는 날짜가 있음
        df = fill_null(df) #데이터프레임 날짜별로 묶고 없는날짜 확인해서 채우기 + 결측치 시간대로 보간하기
    else:
        df = df.groupby(['base_date']).sum() #날짜별로 묶기

    print(df)

    for i in df.isnull().sum():
        if i != 0 : 
            print('전처리 실패')
    print('전처리 성공')

    return df

# 정상성 판단

In [12]:
#수식으로 정상시계열 판단하기
def adf_test(y):
  flag = True
  y = pd.to_numeric(y)
  dftest = adfuller(y,autolag='AIC')
  
  if dftest[1] <= 0.05: 
      flag = True
      print(y.name,'는 정상시계열')
  else: 
      flag = False
      print(y.name,'는 정상아님')

  return flag

In [13]:
def make_normal(srs:pd.Series):
    # srs = pd.to_numeric(srs)
    srs = np.log1p(srs) #로그변환  
    srs = srs.diff().fillna(method='bfill') #1 차분 적용
    return srs

In [14]:
# 처리한 거 정상시계열 맞는지 확인
def check_normal(df:pd.DataFrame):
    for i in df.columns:
        if i == 'em_g': # 타겟제외
            continue
        if not adf_test(df[i]):
            return False
    return True

In [15]:
#main
def adf_main(df:pd.DataFrame):
    for i in df.columns:
        if i == 'em_g':
            continue

        if not adf_test(df[i]): #정상시계열 아니면
            df[i] = make_normal(df[i]) # 정상시계열로 만들고 ( log + diff )
        else: #정상시계열이면
            continue
    print('\n------------ 전처리 후 ------------\n')
    if not check_normal(df):
        print('\n------------ 정상화 실패 ------------')
    else:
        print('\n------------ 정상화 성공 ------------')

    return df

# 제공데이터 불러오기

In [16]:
em = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV',encoding='cp949')
df4 = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/04_음식관련 카드소비_CARD_SPENDING.CSV',encoding='cp949')
df21 = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/02-1_내국인유동인구_KOREAN.CSV',encoding='cp949').drop(['time','emd_cd','age'],axis=1)
df22 = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.CSV',encoding='cp949')
df23 = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/02-3_단기체류 외국인 유동인구_SHORT_TERM_FRGN.CSV',encoding='cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
# df3은 월간데이터 -> 따로처리
df3 = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/01_제공데이터_update(210811)/03_거주인구_RESIDENT_POP.CSV',encoding='cp949')

In [18]:
def proc_df3(df,name):
    tmp_df = df3[df3['emd_nm']==name]

    tmp_df_date_list = []
    for i in range(len(tmp_df)):
        month = tmp_df.iloc[i,:]['base_month']
        if month not in [10,11,12]:
            base_month = '0'+str(month)
        else: base_month = str(month)
        tmp_df_date_list.append(str(tmp_df.iloc[i,:]['base_year'])+base_month)

    real_df3 = tmp_df.drop(['base_year','base_month'],axis=1)
    real_df3['date'] = tmp_df_date_list
    real_df3 = real_df3.groupby('date').sum().reset_index()

    #원래 날짜데이터만큼 확장
    temp_df = pd.DataFrame()
    temp_df['base_date'] = dates
    real_df3_idx = 0
    resid_reg_pop = []
    foreign_pop = []
    total_pop = []

    for i in range(len(temp_df)):
        idx = list(real_df3['date']).index(str(temp_df.iloc[i,:]['base_date'])[:-3].replace('-',''))
        resid_reg_pop.append(real_df3['resid_reg_pop'][idx])
        foreign_pop.append(real_df3['foreign_pop'][idx])
        total_pop.append(real_df3['total_pop'][idx])

    temp_df['resid_reg_pop'] = resid_reg_pop
    temp_df['foreign_pop'] = foreign_pop
    temp_df['total_pop'] = total_pop
    print(temp_df.isnull().sum())
    temp_df = temp_df.set_index('base_date')

    return temp_df

# proc_df3(df3,'노형동')

In [19]:
#제공데이터 main 파이프라인
def final_main(name:str):
    res0 = main(em,name)
    adf_df0 = adf_main(res0)   

    res4 = main(df4,name)
    adf_df4 = adf_main(res4)
    
    res21 = main(df21,name)
    adf_df21 = adf_main(res21)
    
    res22 = main(df22,name)
    adf_df22 = adf_main(res22).drop(['time','emd_cd'],axis=1)
    
    res23 = main(df23,name)
    adf_df23 = adf_main(res23).drop(['time','emd_cd'],axis=1)
    
    adf_df3 = proc_df3(df3,name)
    
    return adf_df0,adf_df4,adf_df21,adf_df22,adf_df23,adf_df3

In [20]:
# colunm 이름 싹 바꾸고
def change_name():
    nm_list = ['adf_df0','adf_df4','adf_df21','adf_df22','adf_df23','adf_df3']
    df_list = [adf_df0,adf_df4,adf_df21,adf_df22,adf_df23,adf_df3]
    for i in range(len(df_list)):
        columns = df_list[i].columns
        real_columns = [nm_list[i][4:]+'_'+j for j in columns]
        df_list[i].columns = real_columns

# raw 데이터 합치기

In [50]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
n_list = []
final = pd.DataFrame()
TRAIN_LEN = int(1277*0.8)
for i in range(42):
    try:
        adf_df0,adf_df4,adf_df21,adf_df22,adf_df23,adf_df3 = final_main(name_list[i])
        change_name()
        F = pd.merge(adf_df0, adf_df4,left_index=True, right_index=True)
        F = pd.merge(F, adf_df21,left_index=True, right_index=True)
        F = pd.merge(F, adf_df22,left_index=True, right_index=True)
        F = pd.merge(F, adf_df23,left_index=True, right_index=True)
        F = pd.merge(F, adf_df3,left_index=True, right_index=True)
        F = F.reset_index()
        base_date = F['base_date'].astype(str)
        # y = F['df0_em_g']

        F = F.drop(['base_date'],axis=1) 
        F = pd.concat([F,base_date],axis=1)
        F = F.set_index('base_date')
        
        file_path = '/content/drive/MyDrive/빅콘2021/Data/'+name_list[i]+'_raw.csv'
        F.to_csv(file_path)
    except:
        if i == 41:
            F = pd.merge(adf_df0, adf_df4,left_index=True, right_index=True)
            F = pd.merge(F, adf_df21,left_index=True, right_index=True)
            F = pd.merge(F, adf_df22,left_index=True, right_index=True)
            F = pd.merge(F, adf_df23,left_index=True, right_index=True)
            F = pd.merge(F, adf_df3,left_index=True, right_index=True)
        n_list.append(i)
    print('###########################################')

print(n_list)

#알수없음 지역은 위 파이프라인을 따라 따로 전처리 => 알수없음_raw.csv

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
resd_pop_cnt 는 정상아님
work_pop_cnt 는 정상아님
visit_pop_cnt 는 정상시계열

------------ 전처리 후 ------------

resd_pop_cnt 는 정상시계열
work_pop_cnt 는 정상시계열
visit_pop_cnt 는 정상시계열

------------ 정상화 성공 ------------
            time       emd_cd  resd_pop_cnt  work_pop_cnt  visit_pop_cnt
base_date                                                               
2018-01-01  3024  12477529440     3669.0126       80.3557      2124.9315
2018-01-02  2771  12176866080     2885.6201       72.2265      1896.1328
2018-01-03  2430  11224765440     2859.4995       97.2012      1660.7993
2018-01-04  3065  12477529440     2613.4210       87.1216      2208.4086
2018-01-05  3771  15033168000     3149.9488      123.6984      2254.3613
...          ...          ...           ...           ...            ...
2021-06-26  5631  22800304800     2185.8190      189.4774      3350.9819
2021-06-27  5539  22349309760     2126.4458       94.1710      3307.0806
2021-06-28  6071  24203400480     2198.2

# 외부데이터 합치기

In [53]:
datafile_list = glob.glob('/content/drive/MyDrive/빅콘2021/Data/*_raw.csv')

# 항공 + 입도객(모든 동 통일)
flightplus = pd.read_csv('/content/drive/MyDrive/빅콘2021/Data/외부데이터/final.csv').drop(['base_date'],axis=1)
# 동별 모든업종 월별 소비금액
total_path_list = sorted(glob.glob('/content/drive/MyDrive/빅콘2021/Data/외부데이터/make_total/*'))
# 동별 여관업 월별 소비금액
room_path_list = sorted(glob.glob('/content/drive/MyDrive/빅콘2021/Data/외부데이터/choose_room/*'))

# TFT용 메타데이터

In [54]:
#TFT
tft = pd.DataFrame()

for i in range(len(datafile_list)):
    df = pd.read_csv(datafile_list[i])
     
    #외부데이터 붙이기
    df = pd.concat([df,flightplus],axis=1)
    reg_name = datafile_list[i][39:-8]
    df['reg'] = reg_name
    tft = pd.concat([tft,df],axis=0,ignore_index=True)

#tft
tmp = tft.iloc[:,1:-1] # base_date, reg 제외
for i in range(len(tmp.columns)):
    tft_scaler = MinMaxScaler()
    scaled_values = tft_scaler.fit_transform(np.array(tmp[tmp.columns[i]]).reshape(-1, 1))
    tmp[tmp.columns[i]] = scaled_values 
    
    if tmp.columns[i] == 'df0_em_g': #역 scaling을 위해 target sclaer만 저장
        print(scaled_values)
        tft_filepath = '/content/drive/MyDrive/빅콘2021/Data/TFT/scaler_tft_emg_minmax.pkl' 
        # joblib.dump(tft_scaler, tft_filepath) 

# tft.to_csv('/content/drive/MyDrive/빅콘2021/Data/TFT/minmax.csv')
tft

[[0.10788652]
 [0.10772621]
 [0.1075659 ]
 ...
 [0.00455457]
 [0.00457985]
 [0.00607155]]


,base_date,df0_em_cnt,df0_em_g,df0_pay_amt,df4_use_cnt,df4_use_amt,df21_resd_pop_cnt,df21_work_pop_cnt,df21_visit_pop_cnt,df22_resd_pop_cnt,df22_work_pop_cnt,df22_visit_pop_cnt,df23_visit_pop_cnt,df3_resid_reg_pop,df3_foreign_pop,df3_total_pop,montly_filght_ld-ar,montly_filght_ld,montly_filght_ar,입도객월계,입도객증감율,reg
0,2018-01-01,-0.001479,1.493500e+06,-0.001487,0.013479,76846214.0,0.004568,22509.9835,191985.3203,34477.9780,4005.5178,9263.3148,-0.180476,20889,3530,24419,24042.000000,24042.000000,24042.000000,1102378,-9.7,한림읍
1,2018-01-02,-0.001479,1.491281e+06,-0.001487,0.013479,85111414.0,0.004568,42942.1307,172570.3144,35704.6224,4851.8636,8710.1331,-0.180476,20889,3530,24419,24042.000000,24042.000000,24042.000000,1102378,-9.7,한림읍
2,2018-01-03,-0.001482,1.489062e+06,-0.001489,-0.096881,65683623.0,0.010769,43510.2829,185352.5570,35952.1894,4166.1843,8914.7940,0.047121,20889,3530,24419,24042.000000,24042.000000,24042.000000,1102378,-9.7,한림읍
3,2018-01-04,-0.001484,1.486842e+06,-0.001491,0.021636,69997698.0,-0.020441,44502.2454,203832.5259,34235.5289,3511.6837,8791.9616,-0.435900,20889,3530,24419,24042.000000,24042.000000,24042.000000,1102378,-9.7,한림읍
4,2018-01-05,-0.001486,1.484623e+06,-0.001494,-0.089503,64775077.0,0.002865,38430.8924,195730.9724,31928.4281,2884.9397,8113.1010,-0.053484,20889,3530,24419,24042.000000,24042.000000,24042.000000,1102378,-9.7,한림읍
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53629,2021-06-26,3.484907,5.785000e+04,85.458186,195.000000,6249373.0,-2377.380341,3846.8010,77757.6622,673.3596,200.1972,4499.9857,70.393645,3348,280,3628,8329.756098,8329.756098,8329.756098,1138867,31.9,알수없음
53630,2021-06-27,1.397895,4.845000e+04,-274.719303,240.000000,8818395.0,1566.180591,3460.5599,66542.9124,778.8933,184.9945,3471.9245,263.019948,3348,280,3628,8329.756098,8329.756098,8329.756098,1138867,31.9,알수없음
53631,2021-06-28,5.639057,6.305000e+04,443.543273,175.000000,4881867.0,2490.394049,4177.8502,65119.6131,695.0088,173.6681,3567.0472,-202.248657,3348,280,3628,8329.756098,8329.756098,8329.756098,1138867,31.9,알수없음
53632,2021-06-29,-0.602105,6.340000e+04,18.549083,205.000000,8586410.0,-510.907905,4725.4219,78257.2063,803.5921,156.9311,8387.3439,-115.308785,3348,280,3628,8329.756098,8329.756098,8329.756098,1138867,31.9,알수없음


# darnn용 학습데이터

In [58]:
#darnn
TRAIN_LEN = int(1277*0.8)

for i in range(len(datafile_list)):
    res = pd.DataFrame()
    df = pd.read_csv(datafile_list[i])
     
    #외부데이터 붙이기
    df = pd.concat([df,flightplus],axis=1).drop(['base_date'],axis=1)
    reg_name = datafile_list[i][39:-8]

    total_path = ''.join([i for i in total_path_list if reg_name in i])
    room_path = ''.join([i for i in room_path_list if reg_name in i])
    total = pd.read_csv(total_path).drop(['base_date'],axis=1)
    room = pd.read_csv(room_path).drop(['base_date'],axis=1)
    if len(room.columns) :
        room.columns = ['월간평균_여관업_소비금액']

    total = pd.concat([total,room],axis=1)
    res = pd.concat([res,df],axis=1)
    res = pd.concat([res,total],axis=1)

    #traintest 분리
    train = res.iloc[:TRAIN_LEN,:]
    test = res.iloc[TRAIN_LEN:,:]

    #피쳐별 민맥스_traintest 분리
    for i in range(len(train.columns)):
        #train
        scaler = MinMaxScaler()
        scaled_values = scaler.fit_transform(np.array(train[train.columns[i]]).reshape(-1, 1))
        train[train.columns[i]] = scaled_values

        #test  
        scaled_values = scaler.transform(np.array(test[test.columns[i]]).reshape(-1, 1))
        test[test.columns[i]] = scaled_values

        if train.columns[i] == 'df0_em_g': #역 scaling을 위해 target sclaer만 저장
            print(scaled_values)
            clip_filepath = '/content/drive/MyDrive/빅콘2021/Data/darnn/scaler__emg_minmax.pkl' 
            # joblib.dump(scaler, clip_filepath) 

        tmp = pd.concat([train,test],axis=0)
        tmp.to_csv('/content/drive/MyDrive/빅콘2021/Data/DARNN/'+reg_name+'_darnn_minmax.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[[0.83141799]
 [0.70563335]
 [0.85295348]
 [0.62308064]
 [0.80056333]
 [0.88879768]
 [0.68475777]
 [0.8395783 ]
 [0.79975857]
 [0.66138741]
 [0.78969902]
 [0.75131177]
 [0.77906004]
 [0.78152261]
 [0.75185901]
 [0.65274425]
 [0.76452599]
 [0.76409142]
 [0.88881378]
 [0.74390793]
 [0.75704169]
 [0.69473684]
 [0.74397232]
 [0.83998069]
 [0.7563174 ]
 [0.74975052]
 [0.77928537]
 [0.73502334]
 [0.84186383]
 [0.734798  ]
 [0.76307742]
 [0.81664252]
 [0.70939965]
 [0.88359891]
 [0.83642363]
 [0.69084178]
 [0.66114598]
 [0.70558506]
 [0.69348141]
 [0.76356028]
 [0.67091582]
 [0.76067922]
 [0.80555287]
 [0.75039433]
 [0.70939965]
 [0.57762756]
 [0.74371479]
 [0.67638822]
 [0.79713504]
 [0.64035088]
 [0.63769516]
 [0.74371479]
 [0.65110253]
 [0.73466924]
 [0.72068244]
 [0.76644133]
 [0.77761146]
 [0.57595365]
 [0.81504909]
 [0.65258329]
 [0.69684532]
 [0.75511025]
 [0.67901175]
 [0.72760341]
 [0.72543055]
 [0.56293256]
 [0.79674875]
 [0.59690971]
 [0.71189441]
 [0.74958957]
 [0.6204732 ]
 [0.71

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[[0.68776431]
 [0.62426429]
 [0.56722762]
 [0.34004011]
 [0.63429176]
 [0.56924402]
 [0.59041069]
 [0.740572  ]
 [0.59857436]
 [0.60063435]
 [0.59719013]
 [0.60224746]
 [0.53710163]
 [0.63429176]
 [0.729302  ]
 [0.58343506]
 [0.58920085]
 [0.56918952]
 [0.62801369]
 [0.60988795]
 [0.63092383]
 [0.66623796]
 [0.59809478]
 [0.58392554]
 [0.5808955 ]
 [0.60744648]
 [0.58117888]
 [0.6259973 ]
 [0.77989711]
 [0.63957797]
 [0.64767624]
 [0.6635349 ]
 [0.60055805]
 [0.57266643]
 [0.69455465]
 [0.63557789]
 [0.67713738]
 [0.62814448]
 [0.62646597]
 [0.620286  ]
 [0.586476  ]
 [0.60941928]
 [0.79162489]
 [0.69044557]
 [0.56590879]
 [0.59408379]
 [0.63473863]
 [0.54940707]
 [0.60777347]
 [0.68345904]
 [0.59194751]
 [0.56718403]
 [0.54394646]
 [0.60634564]
 [0.60602956]
 [0.67373676]
 [0.79247504]
 [0.50828356]
 [0.6071195 ]
 [0.52951563]
 [0.62486376]
 [0.63358329]
 [0.63694031]
 [0.71600471]
 [0.67747526]
 [0.62932162]
 [0.61405153]
 [0.60303222]
 [0.64685879]
 [0.69889262]
 [0.52102498]
 [0.77

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[[0.37432189]
 [0.37110931]
 [0.32408366]
 [0.17203963]
 [0.29517037]
 [0.27913513]
 [0.26309988]
 [0.37731292]
 [0.37709137]
 [0.4021274 ]
 [0.36318862]
 [0.34812269]
 [0.26575858]
 [0.49213525]
 [0.02221117]
 [0.37349105]
 [0.32679775]
 [0.36712127]
 [0.49994516]
 [0.3496182 ]
 [0.38096863]
 [0.35377241]
 [0.36241316]
 [0.37216171]
 [0.33150585]
 [0.42588947]
 [0.37709137]
 [0.37105392]
 [0.36097304]
 [0.39980104]
 [0.43120686]
 [0.39647767]
 [0.28802513]
 [0.37515274]
 [0.36684432]
 [0.51401408]
 [0.39132646]
 [0.29888147]
 [0.25695165]
 [0.33931576]
 [0.3396481 ]
 [0.3748204 ]
 [0.48177743]
 [0.37842071]
 [0.25916723]
 [0.3102363 ]
 [0.42860355]
 [0.34579633]
 [0.43353322]
 [0.48742715]
 [0.36241316]
 [0.35061521]
 [0.46266807]
 [0.4540827 ]
 [0.50559489]
 [0.61781391]
 [0.42334156]
 [0.36479491]
 [0.51368175]
 [0.48764871]
 [0.54215192]
 [0.4950155 ]
 [0.34629483]
 [0.46045249]
 [0.47906334]
 [0.47025642]
 [0.55394988]
 [0.4320377 ]
 [0.38767075]
 [0.51307246]
 [0.44040151]
 [0.63

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

[[ 1.13811599e-01]
 [ 8.07911498e-02]
 [ 8.54844117e-02]
 [ 1.64096547e-01]
 [ 4.41669460e-02]
 [ 1.09704995e-01]
 [ 1.75243044e-01]
 [ 1.41719745e-01]
 [ 9.73851827e-02]
 [ 1.43731143e-01]
 [ 7.81930942e-02]
 [ 2.05581629e-01]
 [ 8.83338921e-02]
 [ 1.80522963e-01]
 [-2.51424740e-04]
 [ 7.70197787e-02]
 [ 1.21605766e-01]
 [ 1.19510560e-01]
 [ 8.53167952e-02]
 [ 5.90848139e-02]
 [ 1.37780758e-01]
 [ 5.95038552e-02]
 [ 7.44217231e-02]
 [ 1.57224271e-01]
 [ 1.04760308e-01]
 [ 2.14213879e-01]
 [ 1.25796178e-01]
 [ 1.21773383e-01]
 [ 8.07073416e-02]
 [ 8.70767684e-02]
 [ 2.52598056e-01]
 [ 1.13224941e-01]
 [ 1.60073751e-01]
 [ 9.57928260e-02]
 [ 1.49849145e-01]
 [ 1.12554475e-01]
 [ 9.19376467e-02]
 [ 2.18236674e-01]
 [ 1.24203822e-01]
 [ 9.04290982e-02]
 [ 6.79684881e-02]
 [ 1.66275562e-01]
 [ 9.29433456e-02]
 [ 9.92289641e-02]
 [ 1.04425075e-01]
 [ 1.70382166e-01]
 [ 1.33506537e-01]
 [ 8.41434797e-02]
 [ 7.75226282e-02]
 [ 9.41166611e-02]
 [ 1.98625545e-01]
 [ 9.93965806e-02]
 [ 8.7412001

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [59]:
tmp

,df0_em_cnt,df0_em_g,df0_pay_amt,df4_use_cnt,df4_use_amt,df21_work_pop_cnt,df21_visit_pop_cnt,df21_resd_pop_cnt,df22_resd_pop_cnt,df22_work_pop_cnt,df22_visit_pop_cnt,df23_visit_pop_cnt,df3_resid_reg_pop,df3_foreign_pop,df3_total_pop,montly_filght_ld-ar,montly_filght_ld,montly_filght_ar,입도객월계,입도객증감율
0,0.385042,0.667365,0.391109,0.741433,0.183563,0.149305,0.529028,0.421870,0.445629,0.052075,0.163832,0.446520,0.925697,1.000000,0.951574,0.870118,0.870118,0.870118,0.660205,0.635538
1,0.384676,0.453905,0.391094,0.595016,0.761052,0.304284,0.356916,0.421869,0.521502,0.306586,0.203436,0.446526,0.925697,1.000000,0.951574,0.870118,0.870118,0.870118,0.660205,0.635538
2,0.345682,0.504274,0.580128,0.454829,0.087276,0.273040,0.313636,0.403664,0.436002,0.216965,0.124660,0.435506,0.925697,1.000000,0.951574,0.870118,0.870118,0.870118,0.660205,0.635538
3,0.359468,0.484495,0.529900,0.411215,0.129443,0.192708,0.313983,0.361257,0.529464,0.231526,0.217399,0.417833,0.925697,1.000000,0.951574,0.870118,0.870118,0.870118,0.660205,0.635538
4,0.458983,0.422980,0.499942,0.716511,0.250046,0.192258,0.310831,0.596240,0.665390,0.282757,0.366450,0.441887,0.925697,1.000000,0.951574,0.870118,0.870118,0.870118,0.660205,0.635538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,0.398164,0.095625,0.547031,0.448598,0.144541,0.459465,0.650874,0.361149,0.172851,0.089095,0.137728,0.438502,-0.219814,-0.210526,-0.222760,0.599068,0.599068,0.599068,0.698959,1.175097
1273,0.382491,0.079869,0.532628,0.588785,0.213187,0.374825,0.535791,0.593156,0.204112,0.082329,0.101306,0.442704,-0.219814,-0.210526,-0.222760,0.599068,0.599068,0.599068,0.698959,1.175097
1274,0.414341,0.104341,0.561351,0.386293,0.108000,0.532011,0.521186,0.647529,0.179264,0.077288,0.104676,0.432554,-0.219814,-0.210526,-0.222760,0.599068,0.599068,0.599068,0.698959,1.175097
1275,0.367471,0.104928,0.544355,0.479751,0.206988,0.652004,0.656001,0.470957,0.211428,0.069840,0.275446,0.434451,-0.219814,-0.210526,-0.222760,0.599068,0.599068,0.599068,0.698959,1.175097
